#**Google Analytics (GA4) Data API**

Этот скрипт запрашивает отчет GA4 у Google Analytics Data API и возвращает его в виде pandas dataframe.

Последовательность действий:

1. Создать проект в Google Cloud Platform в Google-аккаунте, который относится к вертикали клиента Риалвеб
https://cloud.google.com/resource-manager/docs/creating-managing-projects



2.  Активировать возможность использования Google Analytics Data API для этого проекта в Google Cloud Platform: кнопка "+ENABLE APIS AND SERVICES" в разделе APIs & Services, поиск и переход в Google Analytics Data API, кнопка "Enable" 
https://developers.google.com/analytics/devguides/reporting/core/v4

3. Создать сервисный аккаунт, связанный с ранее созданным проектом (кнопка "+CREATE CREDENTIALS" в разделе "CREDENTIALS", выбрать поле "Create service account), выглядит как адрес алектронной почты, например,   ga-api-client@aqueous-thought-372224.iam.gserviceaccount.com)

4. Создать ключ API  для сервисного аккаунта в виде JSON-файла
https://cloud.google.com/iam/docs/creating-managing-service-account-keys

5. В панели управления "Администратор" кабинета Google Analytics в настройках аккаунта или ресурса дайте вашему сервисному аккаунту доступ "Читатель" (в случае если кабинет принадлежит клиенту Риалвеб, выдачу доступа "Читатель" для сервисного аккаунта необходимо запросить у клиента, предоставив ему имя сервисного аккаунта)

6. Используя набор метрик (metrics) и измерений (dimensions), выполнить запрос в соответствии с кодом ниже. Список возможных метрик и измерений: https://developers.google.com/analytics/devguides/reporting/data/v1/api-schema

Примечание: если сервисный аккаунт и проект клиента Риалвеб с активированным Google Analytics Data API уже были созданы в рамках Google Cloud Platform вертикали, то начинать следует с создания собственного ключа API (п.4)

###**Инициализация библиотек и параметров подключения**

In [ ]:
import numpy as np
import pandas as pd
import os
import datetime
from typing import List
!pip install google.analytics.data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (Dimension, Metric, DateRange, Metric, OrderBy, 
                                               FilterExpression, MetricAggregation, CohortSpec, Pivot)
from google.analytics.data_v1beta.types import RunReportRequest, RunRealtimeReportRequest,RunPivotReportRequest

In [ ]:
#Подключаем GDrive и указываем путь к папке с ключем API

from google.colab import drive
drive.mount('/content/drive')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/drive/MyDrive/Colab_Notebooks/zarplata_ru_api_ga.json'

Mounted at /content/drive


ВАЖНО: не публикуйте и не передавайте третьим лицам ваш ключ API, в противном случае возможен несанкционированный доступ к данным, содержащимся в кабинете Google Analitycs, со стороны третьих лиц

In [ ]:
# Указываем номер ресурса GA4 и инициализируем параметры подключения
property_id = '347635549'

client = BetaAnalyticsDataClient()

###**Блок по формированию Run report (стандартный отчет GA4 по метрикам и измерениям)**

Объявляем набор функций необходимых для получения отчета в формате pandas dataframe

In [ ]:
def format_report(request):
    response = client.run_report(request)
    
    # Row index
    row_index_names = [header.name for header in response.dimension_headers]
    row_header = []
    for i in range(len(row_index_names)):
        row_header.append([row.dimension_values[i].value for row in response.rows])

    row_index_named = pd.MultiIndex.from_arrays(np.array(row_header), names = np.array(row_index_names))
    # Row flat data
    metric_names = [header.name for header in response.metric_headers]
    data_values = []
    for i in range(len(metric_names)):
        data_values.append([row.metric_values[i].value for row in response.rows])

    output = pd.DataFrame(data = np.transpose(np.array(data_values, dtype = 'f')), 
                          index = row_index_named, columns = metric_names)
    return output

Запрос отчета Run report

In [ ]:
request = RunReportRequest(
        property='properties/'+property_id,
        dimensions=[Dimension(name="city"),
                    Dimension(name="country"),
                    Dimension(name="browser")],
        metrics=[#Metric(name="averageSessionDuration"),
                 #Metric(name="newUsers"),
                 #Metric(name="activeUsers"),
                 #Metric(name="sessions"),
                 Metric(name="eventCount")],
        date_ranges=[DateRange(start_date="2022-12-25", end_date="today")]
    )
request

property: "properties/347635549"
dimensions {
  name: "city"
}
dimensions {
  name: "country"
}
dimensions {
  name: "browser"
}
metrics {
  name: "eventCount"
}
date_ranges {
  start_date: "2022-12-25"
  end_date: "today"
}

In [ ]:
df = format_report(request)

In [ ]:
df

,,,eventCount
city,country,browser,
Almaty,Kazakhstan,Chrome,172.0
Yerevan,Armenia,Chrome,75.0
Almaty,Kazakhstan,Safari,27.0
Saint Petersburg,Russia,Chrome,20.0
Moscow,Russia,Chrome,17.0
Yekaterinburg,Russia,Safari,17.0
Riga,Latvia,Chrome,16.0
Novosibirsk,Russia,Safari,15.0


### **Блок по формированию Run Pivot report**

Подробнее о Run Pivot report по ссылке: https://developers.google.com/analytics/devguides/reporting/data/v1/pivots

Объявляем набор функций необходимых для получения отчета в формате pandas dataframe

In [ ]:
def format_pivot_report(request):
    response = client.run_pivot_report(request)
    
    # Row index
    row_index_names = [header.name for header in response.dimension_headers]
    row_header = []
    for i in range(len(row_index_names)):
        row_header.append([row.dimension_values[i].value for row in response.rows])

    row_index_named = pd.MultiIndex.from_arrays(np.array(row_header), names = np.array(row_index_names))
    # Row flat data
    metric_names = [header.name for header in response.metric_headers]
    data_values = []
    for i in range(len(metric_names)):
        data_values.append([row.metric_values[i].value for row in response.rows])

    output = pd.DataFrame(data = np.transpose(np.array(data_values, dtype = 'f')), 
                          index = row_index_named, columns = metric_names)
    return output

Формируем pivot_request, где указываем интересующие нас метрики, измерения, пивоты и диапазон дат отчета

In [ ]:
pivot_request = RunPivotReportRequest(
        property='properties/'+property_id,
        dimensions=[Dimension(name="country"),
                    Dimension(name="city"),
                    Dimension(name="date")],
        metrics=[#Metric(name="averageSessionDuration"),
                 #Metric(name="newUsers"),
                 #Metric(name="activeUsers"),
                 Metric(name="sessions"),
                 Metric(name="eventCount")],
        date_ranges=[DateRange(start_date="2022-12-25", end_date="today")],
        pivots = [Pivot(field_names =("city","country","date"), limit = 10)]
    )
pivot_request

Отправляем ранее сформированный запрос, получаем отчет GA4 и формируем из него pandas dataframe

In [ ]:
response = client.run_pivot_report(pivot_request)
df = format_pivot_report(pivot_request)
df

###**Блок по формированию Run Realtime report**

Подробнее о Run Realtime report: https://support.google.com/analytics/answer/9271392?hl=en 

Объявляем необходимые классы и функции

In [ ]:
class GA4RealTimeReport:
   
    def __init__(self, property_id):
        self.property_id = property_id
        self.client = BetaAnalyticsDataClient()

    def query_report(self, dimensions: List[str], metrics: List[Metric], row_limit:int=10000, quota_usage:bool=False):
        
        try:
            dimension_list = [Dimension(name=dim) for dim in dimensions]
            metrics_list = [Metric(name=m) for m in metrics]
            
            report_request = RunRealtimeReportRequest(
                property=f'properties/{self.property_id}',
                dimensions=dimension_list,
                metrics=metrics_list,
                limit=row_limit,
                return_property_quota=quota_usage
            )
            response = self.client.run_realtime_report(report_request)
     
            output = {}
            if 'property_quota' in response:
                output['quota'] = response.property_quota

            # construct the dataset
            headers = [header.name for header in response.dimension_headers] + [header.name for header in response.metric_headers]
            rows = []
            for row in response.rows:
                rows.append(
                    [dimension_value.value for dimension_value in row.dimension_values] + \
                    [metric_value.value for metric_value in row.metric_values])            
            output['headers'] = headers
            output['rows'] = rows
            return output
        except Exception as e:
            raise GA4Exception(e)

Составляем список интересующих метрик и измерений отчета

Полный список доступных метрик и измерений для Run Realtime report представлен по ссылке: https://developers.google.com/analytics/devguides/reporting/data/v1/realtime-api-schema 

In [ ]:
lst_dimensions = ['country','deviceCategory']
lst_metrics = ['activeUsers']

Отправляем запрос, получаем отчет GA4 и формируем из него pandas dataframe

In [ ]:
ga4_realtime = GA4RealTimeReport(property_id)
response = ga4_realtime.query_report(lst_dimensions, lst_metrics, 10, True)

df = pd.DataFrame(data=response['rows'], columns = response['headers'])
df